# Mongo Updater

You don't need to nor can you use this, it's internal to the DBA of the Mongo instance for updating Mongo to match the Repo files. 

In [1]:
from glob import glob
import json
from pymongo import MongoClient
import os
import pandas as pd
import re
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

secrets = json.load(open('../secrets.json'))

client = MongoClient(host='aurii.us', port=20180, username=secrets['MongoUser'], password=secrets['MongoPass'])


pokedex_path = '../Version20/Pokedex'
abilities_path = '../Version20/Abilities'
moves_path = '../Version20/Moves'
learnsets_path = '../Version20/Learnsets'
natures_path = '../Version20/Natures'
items_path = '../Version20/Items'

/Users/bill/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def chunk_data(bulk, x=500):
    ret = []
    for i in range(0,len(bulk),x):
        ret.append(bulk[i:i+x])
    return ret

def publish(dataset, path='', fname_key='', chunk_size=0, mongo=None):
    if mongo is not None: mongo.delete_many({})
    if chunk_size:
        chunks = []
        for i in range(0,len(dataset),chunk_size):
            chunks.append(dataset[i:i+chunk_size])
    else: chunks = dataset
    
    for i in range(len(chunks)):
        if path: 
            fname = chunks[i][fname_key] if fname_key else f"{os.path.basename(path)}{i}"
            open(path+f"/{fname}.json",'w').write(json.dumps(chunks[i], indent=4))
        if mongo is not None: 
            if type(chunks[i]) == list: mongo.insert_many(chunks[i])
            else: mongo.insert_one(chunks[i])

In [3]:
db = client['Pokerole20']

# Pokedex

data = []
for src in sorted(glob(pokedex_path+"/*.json")):
    data.append(json.loads(open(src).read()))
publish(data, fname_key="Name", mongo=db.Pokedex)

# Abilities

data = []
for src in sorted(glob(abilities_path+"/*.json")):
    data.append(json.loads(open(src).read()))
publish(data, fname_key="Name", mongo=db.Abilities)

# Moves

data = []
for src in sorted(glob(moves_path+"/*.json")):
    data.append(json.loads(open(src).read()))
publish(data, fname_key="Name", mongo=db.Moves)

# Learnsets

data = []
for src in sorted(glob(learnsets_path+"/*.json")):
    data.append(json.loads(open(src).read()))
publish(data, fname_key="Name", mongo=db.Learnsets)

# Natures

data = []
for src in sorted(glob(natures_path+"/*.json")):
    data.append(json.loads(open(src).read()))
publish(data, fname_key="Name", mongo=db.Natures)

# Items

data = []
for src in sorted(glob(items_path+"/*.json")):
    data.append(json.loads(open(src).read()))
publish(data, fname_key="Name", mongo=db.Items)